In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import pymongo
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from IPython.display import clear_output

In [2]:
def create_ListofDict(url,view,totalshr,shrFb,shrTw,shrGp,txt):
    info = {
            "URL":url,
            "totalView": view,
            "totalShare": totalshr,
            "share_fb": shrFb,
            "share_tw": shrTw,
            "share_gp": shrGp,
            "text": txt
    }
    return info

In [3]:
def create_ListofDict_InsertMany(df):
    listofdict = []
    for c,lx in enumerate(df.URL,0):
        info = {
            "URL":lx,
            "totalView": int(df.totalView[c]),
            "totalShare": int(df.totalShare[c]),
            "share_fb": int(df.share_fb[c]),
            "share_tw": int(df.share_tw[c]),
            "share_gp": int(df.share_gp[c]),
            "text": df.text[c]
        }
        listofdict.append(info)
    return listofdict

In [4]:
def convertKtoNum(s):
    if  'K' in s:
        ss = s.split('K')
        n = float(ss[0])*1000
    else:
        n = int(s)
    return n

In [5]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
q_c_urlList = mydb["URL_LIST_CRIME"]
q_p_urlList = mydb["URL_LIST_POLITIC"]
q_e_urlList = mydb["URL_LIST_ENTER"]
q_p2_urlList = mydb["POLITIC_NEWS_U"]

i_c_scrape = mydb["CRIME_NEWS"]
i_p_scrape = mydb["POLITIC_NEWS_U"]
i_e_scrape = mydb["ENTERTAIN_NEWS"]

### ข่าวอาชญากรรม

In [ ]:
cursor1 = q_c_urlList.find()

In [ ]:
df_crime = pd.DataFrame(cursor1)
df_crime.tail()

In [ ]:
print('ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)')
g = datetime.strptime(str(max(df_crime.newsdate)), '%Y-%m-%d %H:%M:%S')
mts = timedelta(days=90)
print('Min time:',min(df_crime.newsdate))
print('Max time',max(df_crime.newsdate))
print('Min filter time',g-mts)

In [ ]:
df_crime = df_crime[df_crime.newsdate >= g-mts]
df_crime.head()

df_crime.URL[0:100]  0-99<br>
df_crime.URL[100:300] 100-299

In [ ]:
i_c_scrape.delete_many({})

#df_crime.URL[0:2]

for c,each in enumerate(df_crime.URL,0):
    print('URL',c+1)
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(each)
    try:
        myElem = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CLASS_NAME, 'reader')))
        content=driver.page_source
        soup=BeautifulSoup(content,'lxml')
        div = soup.find("div", class_=lambda value: value and value=="share-box-mobile")
        h2 = div.find("h2", class_=lambda value: value and value=="heading")
        share_a_fb = div.find("a", class_=lambda value: value and value=="share-btn fb", href=True)
        share_a_tw = div.find("a", class_=lambda value: value and value=="share-btn tw", href=True)
        share_a_gp = div.find("a", class_=lambda value: value and value=="share-btn gp", href=True)
        div2 = soup.find("div", class_=lambda value: value and value=="article-subbox")
        view_div = div2.find("div", class_=lambda value: value and value=="upper")
        div3 = soup.find("div", class_=lambda value: value and value=="entry textbox content-all")
        total_share = h2.text.split()[2]
        share_fb = share_a_fb.text.strip()
        share_tw = share_a_tw.text.strip()
        share_gp = share_a_gp.text.strip()
        view_ = int(view_div.text.strip().split()[1].replace(',',''))
        text_  = div3.text.strip()
        process_data = create_ListofDict(each,view_,int(total_share),int(share_fb),int(share_tw),int(share_gp),text_)
        i_c_scrape.insert_one(process_data)
    except TimeoutException:
        print('Request time out')
    driver.quit()
    clear_output(wait=True)

### ข่าวการเมือง

In [6]:
cursor2 = q_p_urlList.find()

In [7]:
df_politic_url = pd.DataFrame(cursor2)
#df_politic_url.tail(3)

In [8]:
print('ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)')
g = datetime.strptime(str(max(df_politic_url.newsdate)), '%Y-%m-%d %H:%M:%S')
mts = timedelta(days=90)
print('Min filter',g-mts)
print('Min time:',min(df_politic_url.newsdate))
print('Max time',max(df_politic_url.newsdate))

ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)
Min filter 2020-08-09 22:15:00
Min time: 2020-08-07 14:50:00
Max time 2020-11-07 22:15:00


In [9]:
df_politic_url = df_politic_url[df_politic_url.newsdate >= g-mts]
df_politic_url.drop_duplicates(subset=['URL'], inplace=True, ignore_index=True)

cursorp2 = q_p2_urlList.find()
df_politic_exDetail = pd.DataFrame(cursorp2)
df_politic_exDetail.drop_duplicates(subset=['URL'], inplace=True, ignore_index=True)

df_politic_j = pd.merge(df_politic_url, df_politic_exDetail, how="left",on=['URL'])
df_politic_j = df_politic_j[df_politic_j._id_y.isnull()]

In [12]:
df_politic_j.head(2)

,_id_x,timestamp,URL,headline,newsdate,_id_y,totalView,totalShare,share_fb,share_tw,share_gp,text
240,5fa6cc5452e52341a96bec20,2020-11-07 23:33:16.231,https://www.dailynews.co.th/politics/804489,"""นพ.วรงค์"" ปลื้มปีติเผย ""ในหลวง"" ทรงตรัส ""ต้อง...",2020-11-02 00:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1312,5fa6d1f752e52341a96bf050,2020-11-07 23:57:19.971,https://www.dailynews.co.th/politics/800489,"""ในหลวง""โปรดเกล้าฯ รมว.คลังเข้าเฝ้าถวายสัตย์ฯ",2020-10-11 21:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
print('Min time:',min(df_politic_j.newsdate))
print('Max time',max(df_politic_j.newsdate))

Min time: 2020-08-09 23:26:00
Max time 2020-11-02 00:54:00


In [11]:
df_politic_j.shape

(2020, 12)

In [14]:
#i_p_scrape.delete_many({})

reqTo = []

for c,each in enumerate(df_politic_j.URL,0):
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(each)
    try:
        myElem = WebDriverWait(driver,4).until(EC.visibility_of_element_located((By.CLASS_NAME, 'reader')))
        content=driver.page_source
        soup=BeautifulSoup(content,'lxml')
        div = soup.find("div", class_=lambda value: value and value=="share-box-mobile")
        h2 = div.find("h2", class_=lambda value: value and value=="heading")
        share_a_fb = div.find("a", class_=lambda value: value and value=="share-btn fb", href=True)
        share_a_tw = div.find("a", class_=lambda value: value and value=="share-btn tw", href=True)
        share_a_gp = div.find("a", class_=lambda value: value and value=="share-btn gp", href=True)
        div2 = soup.find("div", class_=lambda value: value and value=="article-subbox")
        view_div = div2.find("div", class_=lambda value: value and value=="upper")
        div3 = soup.find("div", class_=lambda value: value and value=="entry textbox content-all")
        total_share = h2.text.split()[2]
        share_fb = share_a_fb.text.strip()
        share_tw = share_a_tw.text.strip()
        share_gp = share_a_gp.text.strip()
        view_ = int(view_div.text.strip().split()[1].replace(',',''))
        text_  = div3.text.strip()
        print('URL',c+1,'Element',each,view_,convertKtoNum(total_share),share_fb,share_tw,share_gp,text_)
        process_data = create_ListofDict(each,view_,convertKtoNum(total_share),
                                         convertKtoNum(share_fb),convertKtoNum(share_tw),convertKtoNum(share_gp),text_)
        i_p_scrape.insert_one(process_data)
    except TimeoutException:
        print('Request time out at row',c)
        reqTo.append(c)
    driver.quit()
    clear_output(wait=True)

URL 2020 Element https://www.dailynews.co.th/politics/789088 3700 22 7 7 8 เมื่อวันที่ 9 ส.ค. น.อ.อนุดิษฐ์ นาครทรรพ เลขาธิการพรรคเพื่อไทย พร้อมด้วยนายอนุสรณ์ เอี่ยมสะอาด โฆษกพรรค นายณรงค์ รุ่งธนวงศ์ กรรมการบริหารพรรค นายประเสริฐ ชัยกิจเด่นนภาลัย ผู้อำนวยการเลือกตั้ง และนางสลิลทิพย์ สุขวัฒน์ ผู้สมัครรับเลือกตั้งซ่อม ส.ส.สมุทรปราการ เขต 5  แถลงข่าวที่ศูนย์ประสานงานการเลือกตั้ง เขต 5 จังหวัดสมุทรปราการ เพื่อขอบคุณทุกคะแนนเสียงของพี่น้องชาวสมุทรปราการที่มอบให้กับผู้สมัครของพรรคเพื่อไทย
 
น.อ.อนุดิษฐ์ กล่าวว่า พรรคเพื่อไทยขอขอบคุณทุกคะแนนเสียงของพี่น้องชาวสมุทรปราการที่มอบให้ผู้สมัครของพรรคเพื่อไทยในการเลือกตั้งครั้งนี้ และเชื่อว่าต่อไปในอนาคตพี่น้องจะยังคงให้โอกาสพรรคเพื่อไทยต่อไป อย่างไรก็ตาม ต้องเรียกร้องไปยังคณะกรรมการการเลือกตั้ง (กกต.) ให้ตรวจสอบกรณีที่พบสิ่งผิดปกติต่างๆ โดยพรรคเพื่อไทยจะดำเนินการรวบรวมหลักฐานเพื่อนำเข้าสู่กระบวนการตรวจสอบเพื่อให้การเลือกตั้งครั้งนี้บริสุทธิ์โปร่งใสที่สุด

สำหรับกรณีที่พบว่าหีบบัตรเลือกตั้งหลายแห่ง ไม่เป็นไปตามที่ระเบียบกำหนด เช่น หีบบัตรไม่มีสายรัด ร

In [15]:
reqTo

[0, 1, 837, 1901]

In [ ]:
# Remove duplicate and reinsert POLITIC_NEWS_U แล้วลบ POLITIC_NEWS เดิมออก เพื่อประหยัดเนื้อที่
#i_pp_scrape = mydb["POLITIC_NEWS_U"]
#cc = i_p_scrape.find()
#df_p_u = pd.DataFrame(cc)
#df_p_u = df_p_u.drop_duplicates(subset=['URL'],ignore_index=True)
#df_p_u.shape

In [ ]:
#df_p_u.tail()

In [ ]:
#i_pp_scrape.delete_many({})
#process_data = create_ListofDict_InsertMany(df_p_u)
#i_pp_scrape.insert_many(process_data)

### ข่าวบันเทิง

In [6]:
cursor3 = q_e_urlList.find()

In [7]:
df_entertain = pd.DataFrame(cursor3)
df_entertain.tail(3)

,_id,timestamp,URL,headline,newsdate
1987,5fa6dfa352e52341a96bfab9,2020-11-08 00:55:38.872,https://www.dailynews.co.th/entertainment/763348,"แอบห่วง""แต้ว-ณฐพร"" หลังเป็นไข้ต่ำๆนานแล้วแต่ไม่ลด",2020-03-17 14:13:00
1988,5fa6dfa352e52341a96bfaba,2020-11-08 00:55:38.872,https://www.dailynews.co.th/entertainment/763337,"เก๋ไก๋มาก! ""ดีเจแมน"" เผยชื่อลูกสาวสุดน่ารัก ""เ...",2020-03-17 13:32:00
1989,5fa6dfa352e52341a96bfabb,2020-11-08 00:55:38.872,https://www.dailynews.co.th/entertainment/763325,"""เชน""เรียกร้องทุกฝ่ายทำเพื่อประเทศ ไม่โทษและฟ้...",2020-03-17 13:30:00


In [8]:
print('ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)')
g = datetime.strptime(str(max(df_entertain.newsdate)), '%Y-%m-%d %H:%M:%S')
mts = timedelta(days=90)
print('Min time:',min(df_entertain.newsdate))
print('Max time',max(df_entertain.newsdate))
print('Min filter time',g-mts)

ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)
Min time: 2020-03-17 13:30:00
Max time 2020-11-07 18:52:00
Min filter time 2020-08-09 18:52:00


In [9]:
df_entertain = df_entertain[df_entertain.newsdate >= g-mts]
df_entertain.tail()

,_id,timestamp,URL,headline,newsdate
1026,5fa6da9752e52341a96bf6f8,2020-11-08 00:34:09.518,https://www.dailynews.co.th/entertainment/789121,น้ำใจ'เปา กิ่งกาญจน์'แม้ไม่ว่าง ยังช่วย'เบ บอล...,2020-08-10 12:40:00
1027,5fa6da9752e52341a96bf6f9,2020-11-08 00:34:09.518,https://www.dailynews.co.th/entertainment/789106,ค่าย'เจนนี่'เลือดไหลไม่หยุด 'เบบอล'โบกมือลาอีกคน,2020-08-10 09:26:00
1028,5fa6da9752e52341a96bf6fa,2020-11-08 00:34:09.518,https://www.dailynews.co.th/entertainment/789096,'ซาร่า'เคลื่อนไหวแล้ว ปมคุณแม่เลี้ยงเดี่ยวป่อง...,2020-08-10 07:00:00
1029,5fa6da9752e52341a96bf6fb,2020-11-08 00:34:09.518,https://www.dailynews.co.th/entertainment/789085,'เนเงิน'ขนเด็กในค่ายทำคลิป ลงTiktokตอกกลับคำดูถูก,2020-08-09 22:20:00
1030,5fa6daa452e52341a96bf6fc,2020-11-08 00:34:20.726,https://www.dailynews.co.th/entertainment/789074,BNK48เปิดตัวจัดเต็ม เธียร์เตอร์สเตจใหม่-สุดคุ้...,2020-08-09 19:56:00


In [10]:
df_entertain.shape

(1031, 5)

In [11]:
i_e_scrape.delete_many({})

reqTo = []

for c,each in enumerate(df_entertain.URL,0):
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(each)
    try:
        myElem = WebDriverWait(driver,4).until(EC.visibility_of_element_located((By.CLASS_NAME, 'reader')))
        content=driver.page_source
        soup=BeautifulSoup(content,'lxml')
        div = soup.find("div", class_=lambda value: value and value=="share-box-mobile")
        h2 = div.find("h2", class_=lambda value: value and value=="heading")
        share_a_fb = div.find("a", class_=lambda value: value and value=="share-btn fb", href=True)
        share_a_tw = div.find("a", class_=lambda value: value and value=="share-btn tw", href=True)
        share_a_gp = div.find("a", class_=lambda value: value and value=="share-btn gp", href=True)
        div2 = soup.find("div", class_=lambda value: value and value=="article-subbox")
        view_div = div2.find("div", class_=lambda value: value and value=="upper")
        div3 = soup.find("div", class_=lambda value: value and value=="entry textbox content-all")
        total_share = h2.text.split()[2]
        share_fb = share_a_fb.text.strip()
        share_tw = share_a_tw.text.strip()
        share_gp = share_a_gp.text.strip()
        view_ = int(view_div.text.strip().split()[1].replace(',',''))
        text_  = div3.text.strip()
        print('URL',c+1,'Element',each,view_,convertKtoNum(total_share),share_fb,share_tw,share_gp,text_)
        process_data = create_ListofDict(each,view_,convertKtoNum(total_share),
                                         convertKtoNum(share_fb),convertKtoNum(share_tw),convertKtoNum(share_gp),text_)
        i_e_scrape.insert_one(process_data)
    except TimeoutException:
        print('Request time out at row',c)
        reqTo.append(c)
    driver.quit()
    clear_output(wait=True)

URL 1031 Element https://www.dailynews.co.th/entertainment/789074 6512 46 20 13 13 บริษัท อินดิเพนเด้นท์อาร์ทิสท์เมเนจเม้นท์ จำกัด หรือ iAM (ไอแอม) ผู้ดูแลบริหารงานศิลปินไอดอลหญิงวง “BNK48” ในประเทศไทยเดินหน้าจัดรอบการแสดงจากเธียร์เตอร์สเตจใหม่สุด จากBNK48 ทั้ง 2 ทีม คือทีมNV  1st “Theater no Megami” และทีม Blll 2nd สเตจ ที่ชื่อว่า “Saishuu Bell ga Naru” ณ BNK48 The Campus ชั้น 4 ศูนย์การค้าเดอะมอลล์บางกะปิ ท่ามกลางมาตรการความปลอดภัยด้านสาธารณสุข


















 
สำหรับทีม BNK48 Team NV จำนวน 16 คน ประกอบด้วย ตาหวาน-อิสราภา ธวัชภักดี (Team NV Captain) ปูเป้-จิรดาภา อินทจักร (Team NV Vice-Captain) ,แบมบู-จณิสตา ตันศิริ ,ฟ้อนด์-ณัฐทิชา จันทรวารีเลขา ,จีจี้-ณัฐกุล พิมพ์ธงชัยกุล ,จำ-ณปภัช วรพฤธานนท์ ,ไข่มุก-วรัทยา ดีสมเลิศ ,มิวนิค-นันท์นภัส เลิศนามเชิดสกุล ,โมบายล์-พิมรภัส ผดุงวัฒนะโชค ,มิวสิค-แพรวา สุธรรมพงษ์ ,น้ำหนึ่ง-มิลิน ดอกเทียน ,น้ำใส-พิชญาภา นาถา ,นิว-ชัญญาภัค นุ่มประสพ ,นาย-ภัทรนรินทร์ เหมือนฤทธิ์ ,อร-พัศชนันท์ เจียจิรโชติ และสตางค์-ตรษา ปรีชาตั้งกิจ










var ads_556151; g